# Web Scrapping by Inspection

In [1]:
#Webscrapping Part1

#Webscrapping is applied to worldometers to extract Covid-19 data

#import libraries
import requests
import lxml.html as lh
import pandas as pd

#Scrape table Cells
url='https://www.worldometers.info/coronavirus/'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under container
container = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = container.xpath('//tr')


In [2]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]


[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]

In [3]:
#Testing
tr_elements[1][1].text

'1,239,124'

In [4]:
tr_elements = container.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for x in tr_elements[0]:
    i+=1
    name=x.text_content()
    print (('%d:"%s"')%(i,name))
    col.append((name,[]))
	
#Since the first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #Td is our j'th row
    Td=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    #if len(Td)!=10:
        #break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for x in Td.iterchildren():
        data=x.text_content()
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

#Testing to check content of col
#print(col)


1:"Country,Other"
2:"TotalCases"
3:"NewCases"
4:"TotalDeaths"
5:"NewDeaths"
6:"TotalRecovered"
7:"ActiveCases"
8:"Serious,Critical"
9:"Tot Cases/1M pop"
10:"Deaths/1M pop"
11:"TotalTests"
12:"Tests/
1M pop
"
13:"Continent"


In [5]:
#To check length of the column are the same 
[len(C) for (title,C) in col]


[461, 461, 461, 461, 461, 461, 461, 461, 461, 461, 461, 461, 461]

In [6]:
#Create the Dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Show sample of the dataframe
df.head()


,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Continent
0,\nNorth America\n,"1,239,124","+14,169","72,830","+1,050","202,001","964,293","17,724",,,,,North America
1,\nEurope\n,"1,422,787","+24,097","139,163","+1,754","530,673","752,951","17,785",,,,,Europe
2,\nAsia\n,"542,707","+8,866","19,203",+242,"282,936","240,568","5,500",,,,,Asia
3,\nSouth America\n,"193,304","+3,205","9,833",+353,"64,716","118,755","9,849",,,,,South America
4,\nOceania\n,"8,370",+22,113,+1,"7,134","1,123",30,,,,,Australia/Oceania


# Insert Into MongoDB

In [18]:
#Import into MongoDB
import pymongo 
from pymongo import MongoClient

mng_client = pymongo.MongoClient('localhost', 27017) # Making a Connection with MongoClient
mng_db = mng_client['WorldometerDatabase'] # Replace mongodb name
collection_name = 'Worldometer' # Replace mongodb collection name
db_cm = mng_db[collection_name]

#Insert Data 
db_cm.insert_one({"index":"World","Web_Data":Dict})


# Retrieve from MongoDB

In [24]:
data_from_db = db_cm.find_one({"index":"World"})
dfmongo = pd.DataFrame(data_from_db["Web_Data"])

dfmongo.head() # view sample of the dataframe

# print(dfmongo) # print the dataframe


,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Continent
0,\nNorth America\n,"1,104,614","+13,818","62,770","+1,221","171,053","870,791","15,412",,,,,North America
1,\nEurope\n,"1,321,890","+20,619","125,915","+1,918","457,982","737,993","24,851",,,,,Europe
2,\nAsia\n,"492,632","+9,406","17,830",+242,"244,923","229,879","5,833",,,,,Asia
3,\nSouth America\n,"147,524","+3,350","7,081",+351,"52,029","88,414","9,773",,,,,South America
4,\nOceania\n,"8,305",+14,103,+1,"6,918","1,284",45,,,,,Australia/Oceania


# Export Worldometer Dataframe to CSV file

In [12]:
#Export Dataframe to CSV file
dfmongo.to_csv (r'C:\Users\presi\export_worldometer.csv', index = False, header=True)

# Web Scrapping by API Call

In [35]:
#Webscrapping Part2

#API Call will be used for this part of webscrapping

#import the needed library
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. 
# Hence, the reason we specicified 'None' to replace inputs for application token, and no username or password:
client = Socrata("data.medicare.gov", None)


# To extract First 2000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
hospdata = client.get("xubh-q36u", limit=2000)

# Convert to pandas DataFrame
hospital_data = pd.DataFrame.from_records(hospdata)

# Show sample of the dataframe
hospital_data.head()

,provider_id,hospital_name,address,city,state,zip_code,county_name,phone_number,hospital_type,hospital_ownership,...,timeliness_of_care_national_comparison_footnote,efficient_use_of_medical_imaging_national_comparison,efficient_use_of_medical_imaging_national_comparison_footnote,readmission_national_comparison_footnote,patient_experience_national_comparison_footnote,effectiveness_of_care_national_comparison_footnote,geocoded_column,:@computed_region_f3tr_pr43,:@computed_region_nwen_78xc,:@computed_region_csmy_5jwy
0,030084,CHINLE COMPREHENSIVE HEALTH CARE FACILITY,"US HWY 191, HOSPITAL ROAD",CHINLE,AZ,86503,APACHE,(928) 674-7001,Acute Care Hospitals,Government - Federal,...,5,Not Available,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,044022,CONWAY BEHAVIORAL HEALTH,2255 STURGIS ROAD,CONWAY,AR,72034,FAULKNER,(501) 205-0011,Psychiatric,Proprietary,...,19,Not Available,19,19,19,19,NaN,NaN,NaN,NaN
2,10021F,96th Medical Group (Eglin AFB),96 MDG 307 Boatner Rd Suite 114,Eglin AFB,FL,32542,OKALOOSA,(850) 883-8600,Acute Care - Department of Defense,Department of Defense,...,22,Not Available,22,22,22,22,NaN,NaN,NaN,NaN
3,170204,"ROCK REGIONAL HOSPITAL, LLC",3251 NORTH ROCK ROAD,DERBY,KS,67037,SEDGWICK,(833) 345-7625,Acute Care Hospitals,Proprietary,...,5,Not Available,5,5,5,5,"{'type': 'Point', 'coordinates': [-97.244407, ...",1291,1291,17
4,171322,HAMILTON COUNTY HOSPITAL,700 NORTH HUSER,SYRACUSE,KS,67878,HAMILTON,(620) 384-7461,Critical Access Hospitals,Government - Local,...,NaN,Not Available,16,16,16,5,NaN,NaN,NaN,NaN


In [36]:
data_dict = hospital_data.to_dict("records")

# Insert into MongoDB

In [39]:
#Import into MongoDB
import pymongo 
from pymongo import MongoClient

mng_client = pymongo.MongoClient('localhost', 27017) # Making a Connection with MongoClient
mng_db = mng_client['WorldometerDatabase'] # Replace mongodb name
collection_name = 'Hospital' # Replace mongodb collection name
hb_cm = mng_db[collection_name]

#Insert Data 
hb_cm.insert_one({"index":"USA","Hospital_Data":data_dict})


# Retrieve from MongoDB

In [40]:
data_from_db2 = hb_cm.find_one({"index":"USA"})
hfmongo = pd.DataFrame(data_from_db2["Hospital_Data"])

hfmongo.head() # view sample of the dataframe

# print(dfmongo) # print the dataframe


,provider_id,hospital_name,address,city,state,zip_code,county_name,phone_number,hospital_type,hospital_ownership,...,timeliness_of_care_national_comparison_footnote,efficient_use_of_medical_imaging_national_comparison,efficient_use_of_medical_imaging_national_comparison_footnote,readmission_national_comparison_footnote,patient_experience_national_comparison_footnote,effectiveness_of_care_national_comparison_footnote,geocoded_column,:@computed_region_f3tr_pr43,:@computed_region_nwen_78xc,:@computed_region_csmy_5jwy
0,030084,CHINLE COMPREHENSIVE HEALTH CARE FACILITY,"US HWY 191, HOSPITAL ROAD",CHINLE,AZ,86503,APACHE,(928) 674-7001,Acute Care Hospitals,Government - Federal,...,5,Not Available,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,044022,CONWAY BEHAVIORAL HEALTH,2255 STURGIS ROAD,CONWAY,AR,72034,FAULKNER,(501) 205-0011,Psychiatric,Proprietary,...,19,Not Available,19,19,19,19,NaN,NaN,NaN,NaN
2,10021F,96th Medical Group (Eglin AFB),96 MDG 307 Boatner Rd Suite 114,Eglin AFB,FL,32542,OKALOOSA,(850) 883-8600,Acute Care - Department of Defense,Department of Defense,...,22,Not Available,22,22,22,22,NaN,NaN,NaN,NaN
3,170204,"ROCK REGIONAL HOSPITAL, LLC",3251 NORTH ROCK ROAD,DERBY,KS,67037,SEDGWICK,(833) 345-7625,Acute Care Hospitals,Proprietary,...,5,Not Available,5,5,5,5,"{'type': 'Point', 'coordinates': [-97.244407, ...",1291,1291,17
4,171322,HAMILTON COUNTY HOSPITAL,700 NORTH HUSER,SYRACUSE,KS,67878,HAMILTON,(620) 384-7461,Critical Access Hospitals,Government - Local,...,NaN,Not Available,16,16,16,5,NaN,NaN,NaN,NaN


# Export Hospital Dataframe to CSV file

In [ ]:
#Export Dataframe to CSV file
hfmongo.to_csv (r'C:\Users\presi\export_hospital_data.csv', index = False, header=True)